In [1]:
print('ねんむい')

ねんむい


In [2]:
from sympy import *
init_printing(use_latex='mathjax') # windowsはカッコ内のmathjax指定必須
from sympy.stats import *

# The model

## Model description
* 1期間
* リスク中立的なプリンシパル
* 2人のリスク回避的なエージェント
 * 各エージェントは1つのタスクを行う
 * エージェントの行動はもうひとりのエージェントやプリンシパルからは観察できない

In [3]:
a1 = Symbol('a_1')
a2 = Symbol('a_2')
aj = Symbol('a_j')
k = Lambda((aj), Rational(1, 2) * (aj ** 2)) # sympy1.5からLambdaを書くときにはリストではなくタプルで指定するらしい
[k(a1), k(a2)]

⎡  2    2⎤
⎢a₁   a₂ ⎥
⎢───, ───⎥
⎣ 2    2 ⎦

In [4]:
m = Symbol('m')
eps1 = Normal('varepsilon_1', 0, 1)
eps2 = Normal('varepsilon_2', 0, 1)
y1 = m*a1 + eps1
y2 = m*a2 + eps2

x = a1 + a2

[y1, y2, E(y1), E(y2), x]

[a₁⋅m + varepsilon₁, a₂⋅m + varepsilon₂, a₁⋅m, a₂⋅m, a₁ + a₂]

In [5]:
f1 = Symbol('f_1') 
f2 = Symbol('f_2')
v11 = Symbol('v_{11}')
v12 = Symbol('v_{12}')
v21 = Symbol('v_{21}')
v22 = Symbol('v_{22}')

c1 = f1 + v11*y1 + v12*y2
c2 = f2 + v21*y1 + v22*y2

[c1, c2, E(c1), E(c2)]

[f₁ + v_{11}⋅(a₁⋅m + varepsilon₁) + v_{12}⋅(a₂⋅m + varepsilon₂), f₂ + v_{21}⋅(
a₁⋅m + varepsilon₁) + v_{22}⋅(a₂⋅m + varepsilon₂), a₁⋅m⋅v_{11} + a₂⋅m⋅v_{12} +
 f₁, a₁⋅m⋅v_{21} + a₂⋅m⋅v_{22} + f₂]

In [6]:
rho = Symbol('rho')
Var1 = variance(c1) + 2*rho*v11*v12
Var2 = variance(c2) + 2*rho*v22*v21
[Var1, Var2]

⎡                          2         2                            2         2⎤
⎣2⋅ρ⋅v_{11}⋅v_{12} + v_{11}  + v_{12} , 2⋅ρ⋅v_{21}⋅v_{22} + v_{21}  + v_{22} ⎦

In [7]:
r = Symbol('r')
CE1 = E(c1) - k(a1) - Rational(1, 2)*r*Var1
CE2 = E(c2) - k(a2) - Rational(1, 2)*r*Var2
[CE1, CE2]

⎡    2                                      ⎛                          2      
⎢  a₁                                     r⋅⎝2⋅ρ⋅v_{11}⋅v_{12} + v_{11}  + v_{
⎢- ─── + a₁⋅m⋅v_{11} + a₂⋅m⋅v_{12} + f₁ - ────────────────────────────────────
⎣   2                                                         2               

   2⎞                  2                        ⎛                          2  
12} ⎠                a₂                       r⋅⎝2⋅ρ⋅v_{21}⋅v_{22} + v_{21}  +
─────, a₁⋅m⋅v_{21} - ─── + a₂⋅m⋅v_{22} + f₂ - ────────────────────────────────
                      2                                           2           

       2⎞⎤
 v_{22} ⎠⎥
─────────⎥
         ⎦

## Optinal linear contracts with global reporting and full commitment

In [8]:
aj1 = solve(CE1.diff(a1), a1)[0]
aj2 = solve(CE2.diff(a2), a2)[0]
[aj1, aj2]

[m⋅v_{11}, m⋅v_{22}]

In [9]:
CE10 = CE1.subs([(a1, aj1)])
CE20 = CE2.subs([(a2, aj2)])

[CE10, CE20]

⎡                    2       2     ⎛                          2         2⎞    
⎢                   m ⋅v_{11}    r⋅⎝2⋅ρ⋅v_{11}⋅v_{12} + v_{11}  + v_{12} ⎠    
⎢a₂⋅m⋅v_{12} + f₁ + ────────── - ─────────────────────────────────────────, a₁
⎣                       2                            2                        

                  2       2     ⎛                          2         2⎞⎤
                 m ⋅v_{22}    r⋅⎝2⋅ρ⋅v_{21}⋅v_{22} + v_{21}  + v_{22} ⎠⎥
⋅m⋅v_{21} + f₂ + ────────── - ─────────────────────────────────────────⎥
                     2                            2                    ⎦

In [10]:
Pi = E(x - (c1 + c2))
Pi

-a₁⋅m⋅v_{11} - a₁⋅m⋅v_{21} + a₁ - a₂⋅m⋅v_{12} - a₂⋅m⋅v_{22} + a₂ - f₁ - f₂

In [11]:
fj1 = solve(CE10, f1)[0]
fj2 = solve(CE20, f2)[0]
[fj1, fj2]

⎡                2       2                               2           2        
⎢               m ⋅v_{11}                        r⋅v_{11}    r⋅v_{12}         
⎢-a₂⋅m⋅v_{12} - ────────── + r⋅ρ⋅v_{11}⋅v_{12} + ───────── + ─────────, -a₁⋅m⋅
⎣                   2                                2           2            

          2       2                               2           2⎤
         m ⋅v_{22}                        r⋅v_{21}    r⋅v_{22} ⎥
v_{21} - ────────── + r⋅ρ⋅v_{21}⋅v_{22} + ───────── + ─────────⎥
             2                                2           2    ⎦

In [12]:
Pi0 = Pi.subs([(f1, fj1), (f2, fj2), (a1, aj1), (a2, aj2)])
Pi0

   2       2    2       2                                                     
  m ⋅v_{11}    m ⋅v_{22}                                                      
- ────────── - ────────── + m⋅v_{11} + m⋅v_{22} - r⋅ρ⋅v_{11}⋅v_{12} - r⋅ρ⋅v_{2
      2            2                                                          

                    2           2           2           2
            r⋅v_{11}    r⋅v_{12}    r⋅v_{21}    r⋅v_{22} 
1}⋅v_{22} - ───────── - ───────── - ───────── - ─────────
                2           2           2           2    

In [13]:
v110 = simplify(solve(Pi0.diff(v11), v11)[0])
v120 = simplify(solve(Pi0.diff(v12), v12)[0])

[v110, v120]

⎡m - r⋅ρ⋅v_{12}           ⎤
⎢──────────────, -ρ⋅v_{11}⎥
⎢     2                   ⎥
⎣    m  + r               ⎦

In [14]:
eq1 = Eq(v110, v11)
eq2 = Eq(v120, v12)
v1dag = solve([eq1, eq2], [v11, v12])
v1dag

⎧              m                      m⋅ρ      ⎫
⎪v_{11}: ─────────────, v_{12}: ───────────────⎪
⎨         2      2                 2      2    ⎬
⎪        m  - r⋅ρ  + r          - m  + r⋅ρ  - r⎪
⎩                                              ⎭

In [15]:
v210 = simplify(solve(Pi0.diff(v21), v21)[0])
v220 = simplify(solve(Pi0.diff(v22), v22)[0])

[v210, v220]

⎡           m - r⋅ρ⋅v_{21}⎤
⎢-ρ⋅v_{22}, ──────────────⎥
⎢                2        ⎥
⎣               m  + r    ⎦

In [16]:
eq3 = Eq(v210, v21)
eq4 = Eq(v220, v22)
v2dag = solve([eq3, eq4], [v21, v22])
v2dag

⎧              m⋅ρ                      m      ⎫
⎪v_{21}: ───────────────, v_{22}: ─────────────⎪
⎨           2      2               2      2    ⎬
⎪        - m  + r⋅ρ  - r          m  - r⋅ρ  + r⎪
⎩                                              ⎭

In [17]:
Pidag = simplify(Pi0.subs([(v11, v1dag[v11]), (v12, v1dag[v12]), (v21, v2dag[v21]), (v22, v2dag[v22])]))
Pidag

       2     
      m      
─────────────
 2      2    
m  - r⋅ρ  + r